In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg

import os.path as op
import time


from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras_tqdm import TQDMNotebookCallback
import numpy as np

from kiki import kiki_net
from data import MaskedUntouched2DSequence
from evaluate import psnr

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 7135
n_samples_val = 7135

n_volumes_train = 199
n_volumes_val = 199

In [4]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [5]:
run_params = {
    'n_cascade': 2, 
    'n_convs': 25, 
    'n_filters': 64,
    'noiseless': True,
}

n_epochs = 10
run_id = f'kikinet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

cascadenet_af4_1566913104


In [6]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_keras_psnr', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [7]:
model = kiki_net(lr=1e-3, **run_params)
print(model.summary(line_length=100))

W0827 15:38:24.867094 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 15:38:24.876030 140591726143232 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:63: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0827 15:38:24.902921 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 15:38:24.910788 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

Model: "model_1"
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
kspace_input (InputLayer)        (None, 640, None, 1)  0                                            
____________________________________________________________________________________________________
ifft_simple (Lambda)             (None, 640, None, 1)  0           kspace_input[0][0]               
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 640, None, 1)  0           ifft_simple[0][0]                
__________________________________________________________________________

In [8]:
# model.fit_generator(
#     train_gen, 
#     steps_per_epoch=n_volumes_train, 
#     epochs=n_epochs,
#     validation_data=val_gen,
#     validation_steps=5,
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
#     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [9]:
%%time
# simple overfit trials
data = train_gen[0]
val_data = val_gen[1]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
#     callbacks=[tboard_cback],
    epochs=500,
    verbose=2, 
    shuffle=False,
)

W0827 15:38:28.220433 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0827 15:38:29.477745 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.

W0827 15:38:34.108402 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0827 15:38:34.112783 140591726143232 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/500
 - 7s - loss: 2.5980e-05 - mean_squared_error: 1.8325e-09 - keras_psnr: 25.8021 - keras_ssim: 0.5126
Epoch 2/500
 - 1s - loss: 2.4375e-05 - mean_squared_error: 1.2979e-09 - keras_psnr: 27.3002 - keras_ssim: 0.4782
Epoch 3/500
 - 1s - loss: 2.3338e-05 - mean_squared_error: 1.2348e-09 - keras_psnr: 27.5164 - keras_ssim: 0.5245
Epoch 4/500
 - 1s - loss: 2.1656e-05 - mean_squared_error: 1.1547e-09 - keras_psnr: 27.8079 - keras_ssim: 0.6058
Epoch 5/500
 - 1s - loss: 2.0147e-05 - mean_squared_error: 1.0685e-09 - keras_psnr: 28.1448 - keras_ssim: 0.6701
Epoch 6/500
 - 1s - loss: 1.8457e-05 - mean_squared_error: 8.7030e-10 - keras_psnr: 29.0359 - keras_ssim: 0.7196
Epoch 7/500
 - 1s - loss: 1.7116e-05 - mean_squared_error: 7.5472e-10 - keras_psnr: 29.6547 - keras_ssim: 0.7491
Epoch 8/500
 - 1s - loss: 1.5284e-05 - mean_squared_error: 5.9666e-10 - keras_psnr: 30.6752 - keras_ssim: 0.7649
Epoch 9/500
 - 1s - loss: 1.4103e-05 - mean_squared_error: 5.0739e-10 - keras_psnr: 31.3791 - ke

Epoch 73/500
 - 1s - loss: 6.9802e-06 - mean_squared_error: 9.2031e-11 - keras_psnr: 38.7932 - keras_ssim: 0.8995
Epoch 74/500
 - 1s - loss: 6.9851e-06 - mean_squared_error: 9.2154e-11 - keras_psnr: 38.7874 - keras_ssim: 0.8992
Epoch 75/500
 - 1s - loss: 6.9652e-06 - mean_squared_error: 9.0837e-11 - keras_psnr: 38.8499 - keras_ssim: 0.9003
Epoch 76/500
 - 1s - loss: 6.9924e-06 - mean_squared_error: 9.2290e-11 - keras_psnr: 38.7810 - keras_ssim: 0.8994
Epoch 77/500
 - 1s - loss: 6.9421e-06 - mean_squared_error: 9.0392e-11 - keras_psnr: 38.8712 - keras_ssim: 0.9009
Epoch 78/500
 - 1s - loss: 6.9630e-06 - mean_squared_error: 9.1388e-11 - keras_psnr: 38.8237 - keras_ssim: 0.9005
Epoch 79/500
 - 1s - loss: 6.9831e-06 - mean_squared_error: 9.3676e-11 - keras_psnr: 38.7163 - keras_ssim: 0.8995
Epoch 80/500
 - 1s - loss: 6.8910e-06 - mean_squared_error: 8.9897e-11 - keras_psnr: 38.8951 - keras_ssim: 0.9009
Epoch 81/500
 - 1s - loss: 6.9574e-06 - mean_squared_error: 9.1001e-11 - keras_psnr: 38.

Epoch 145/500
 - 1s - loss: 6.8979e-06 - mean_squared_error: 8.6804e-11 - keras_psnr: 39.0472 - keras_ssim: 0.9045
Epoch 146/500
 - 1s - loss: 6.7280e-06 - mean_squared_error: 8.1316e-11 - keras_psnr: 39.3308 - keras_ssim: 0.9064
Epoch 147/500
 - 1s - loss: 6.7118e-06 - mean_squared_error: 8.1201e-11 - keras_psnr: 39.3369 - keras_ssim: 0.9066
Epoch 148/500
 - 1s - loss: 6.7102e-06 - mean_squared_error: 8.1968e-11 - keras_psnr: 39.2961 - keras_ssim: 0.9064
Epoch 149/500
 - 1s - loss: 6.7343e-06 - mean_squared_error: 8.1993e-11 - keras_psnr: 39.2948 - keras_ssim: 0.9069
Epoch 150/500
 - 1s - loss: 6.7522e-06 - mean_squared_error: 8.4293e-11 - keras_psnr: 39.1746 - keras_ssim: 0.9048
Epoch 151/500
 - 1s - loss: 6.7554e-06 - mean_squared_error: 8.2240e-11 - keras_psnr: 39.2817 - keras_ssim: 0.9064
Epoch 152/500
 - 1s - loss: 6.7148e-06 - mean_squared_error: 8.1767e-11 - keras_psnr: 39.3067 - keras_ssim: 0.9064
Epoch 153/500
 - 1s - loss: 6.6508e-06 - mean_squared_error: 7.9534e-11 - keras_

Epoch 217/500
 - 1s - loss: 6.9415e-06 - mean_squared_error: 8.8300e-11 - keras_psnr: 38.9729 - keras_ssim: 0.9081
Epoch 218/500
 - 1s - loss: 6.9665e-06 - mean_squared_error: 9.0616e-11 - keras_psnr: 38.8605 - keras_ssim: 0.9073
Epoch 219/500
 - 1s - loss: 6.9132e-06 - mean_squared_error: 8.4071e-11 - keras_psnr: 39.1861 - keras_ssim: 0.9053
Epoch 220/500
 - 1s - loss: 6.6405e-06 - mean_squared_error: 7.8411e-11 - keras_psnr: 39.4888 - keras_ssim: 0.9096
Epoch 221/500
 - 1s - loss: 6.6062e-06 - mean_squared_error: 7.6762e-11 - keras_psnr: 39.5811 - keras_ssim: 0.9109
Epoch 222/500
 - 1s - loss: 6.6043e-06 - mean_squared_error: 7.8008e-11 - keras_psnr: 39.5112 - keras_ssim: 0.9105
Epoch 223/500
 - 1s - loss: 6.7316e-06 - mean_squared_error: 8.1932e-11 - keras_psnr: 39.2980 - keras_ssim: 0.9102
Epoch 224/500
 - 1s - loss: 6.9477e-06 - mean_squared_error: 8.8443e-11 - keras_psnr: 38.9659 - keras_ssim: 0.9054
Epoch 225/500
 - 1s - loss: 7.0162e-06 - mean_squared_error: 8.6497e-11 - keras_

Epoch 289/500
 - 1s - loss: 6.0343e-06 - mean_squared_error: 6.3508e-11 - keras_psnr: 40.4042 - keras_ssim: 0.9217
Epoch 290/500
 - 1s - loss: 5.9735e-06 - mean_squared_error: 6.3243e-11 - keras_psnr: 40.4224 - keras_ssim: 0.9219
Epoch 291/500
 - 1s - loss: 5.9668e-06 - mean_squared_error: 6.2944e-11 - keras_psnr: 40.4430 - keras_ssim: 0.9219
Epoch 292/500
 - 1s - loss: 5.9317e-06 - mean_squared_error: 6.1799e-11 - keras_psnr: 40.5227 - keras_ssim: 0.9225
Epoch 293/500
 - 1s - loss: 5.9440e-06 - mean_squared_error: 6.2657e-11 - keras_psnr: 40.4629 - keras_ssim: 0.9220
Epoch 294/500
 - 1s - loss: 5.9294e-06 - mean_squared_error: 6.1351e-11 - keras_psnr: 40.5543 - keras_ssim: 0.9230
Epoch 295/500
 - 1s - loss: 5.9480e-06 - mean_squared_error: 6.2715e-11 - keras_psnr: 40.4589 - keras_ssim: 0.9225
Epoch 296/500
 - 1s - loss: 5.9202e-06 - mean_squared_error: 6.1601e-11 - keras_psnr: 40.5366 - keras_ssim: 0.9232
Epoch 297/500
 - 1s - loss: 5.9311e-06 - mean_squared_error: 6.2330e-11 - keras_

Epoch 361/500
 - 1s - loss: 5.8442e-06 - mean_squared_error: 6.0713e-11 - keras_psnr: 40.5997 - keras_ssim: 0.9257
Epoch 362/500
 - 1s - loss: 5.7431e-06 - mean_squared_error: 5.8573e-11 - keras_psnr: 40.7555 - keras_ssim: 0.9273
Epoch 363/500
 - 1s - loss: 5.7191e-06 - mean_squared_error: 5.7926e-11 - keras_psnr: 40.8038 - keras_ssim: 0.9273
Epoch 364/500
 - 1s - loss: 5.6866e-06 - mean_squared_error: 5.7109e-11 - keras_psnr: 40.8655 - keras_ssim: 0.9280
Epoch 365/500
 - 1s - loss: 5.7526e-06 - mean_squared_error: 5.8544e-11 - keras_psnr: 40.7577 - keras_ssim: 0.9270
Epoch 366/500
 - 1s - loss: 5.7751e-06 - mean_squared_error: 5.9428e-11 - keras_psnr: 40.6927 - keras_ssim: 0.9261
Epoch 367/500
 - 1s - loss: 5.7684e-06 - mean_squared_error: 5.7866e-11 - keras_psnr: 40.8083 - keras_ssim: 0.9279
Epoch 368/500
 - 1s - loss: 5.8285e-06 - mean_squared_error: 6.0565e-11 - keras_psnr: 40.6104 - keras_ssim: 0.9268
Epoch 369/500
 - 1s - loss: 5.8324e-06 - mean_squared_error: 5.8780e-11 - keras_

Epoch 433/500
 - 1s - loss: 5.9234e-06 - mean_squared_error: 6.0283e-11 - keras_psnr: 40.6306 - keras_ssim: 0.9286
Epoch 434/500
 - 1s - loss: 6.3324e-06 - mean_squared_error: 7.3083e-11 - keras_psnr: 39.7944 - keras_ssim: 0.9215
Epoch 435/500
 - 1s - loss: 6.0122e-06 - mean_squared_error: 6.2003e-11 - keras_psnr: 40.5084 - keras_ssim: 0.9269
Epoch 436/500
 - 1s - loss: 5.8103e-06 - mean_squared_error: 5.8471e-11 - keras_psnr: 40.7632 - keras_ssim: 0.9269
Epoch 437/500
 - 1s - loss: 5.7576e-06 - mean_squared_error: 5.8203e-11 - keras_psnr: 40.7831 - keras_ssim: 0.9275
Epoch 438/500
 - 1s - loss: 5.6757e-06 - mean_squared_error: 5.5614e-11 - keras_psnr: 40.9807 - keras_ssim: 0.9294
Epoch 439/500
 - 1s - loss: 5.8228e-06 - mean_squared_error: 5.9184e-11 - keras_psnr: 40.7105 - keras_ssim: 0.9276
Epoch 440/500
 - 1s - loss: 5.8415e-06 - mean_squared_error: 5.8563e-11 - keras_psnr: 40.7563 - keras_ssim: 0.9297
Epoch 441/500
 - 1s - loss: 6.0445e-06 - mean_squared_error: 6.5044e-11 - keras_